In [1]:
#import all needed libs and set base paths for the data
import pandas as pd, numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

base_path = Path()
raw_data_path = base_path.joinpath('raw-data')
processed_data_path = base_path.joinpath('processed-data')
split_data_path = base_path.joinpath('split-data')

In [51]:
#All functions for handling the data are defined here

def load_data(type:str):
    '''type: raw, processed, or split \n
    returns a dataframe of all the data from the specified type'''
    type = type.lower()
    try:
        if type == 'raw':
            all_files = raw_data_path.glob('*.csv')
        elif type == 'processed':
            all_files = processed_data_path.glob('*.csv')
        elif type == 'split':
            all_files = split_data_path.glob('*.csv')
        all_files = [filename.as_posix() for filename in all_files]

        if len(list(all_files)) == 0:
            print('No files found in raw data folder')
            return None
        elif len(list(all_files)) == 1:
            data = pd.read_csv(all_files[0], encoding_errors='ignore', header=0)
        elif len(list(all_files)) > 1:
            data = pd.concat([pd.read_csv(f, encoding_errors='ignore', header=0) for f in all_files])
        return data
    except Exception as e:
        print(e)
        return None
    
def prune_col(data):
    '''Takes in a dataframe and removes all unnecessary columns.\n
    Returns the dataframe with the columns removed.'''
    keep_columns = [
        "BankName", "GrossApproval", "ApprovalDate", "ApprovalFiscalYear", "InitialInterestRate", "TermInMonths", "NaicsCode", "BusinessType", "BusinessAge", "LoanStatus",
        "PaidInFullDate", "GrossChargeOffAmount", "RevolverStatus", "JobsSupported"
    ]
    data = data[keep_columns]
    return data

def fill_missing(data):
    '''Takes in a pruned dataframe and fills some of the columns with estimated values. \n
    Returns the dataframe with the columns filled. '''

    data['BusinessType'] = data['BusinessType'].fillna('Individual')
    data['GrossApproval'] = data['GrossApproval'].fillna(0)

    min_intrest_map = data['ApprovalFiscalYear', 'InitialInterestRate'].groupby('ApprovalFiscalYear').min().to_dict()
    data['InitialInterestRate'] = 
    
    
    return data


In [52]:
raw_data = load_data('raw')
pruned_data = fill_missing(prune_col(raw_data))
print(pruned_data.shape)
display(pruned_data.head())

C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:22: DtypeWarning: Columns (13,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv(f, encoding_errors='ignore', header=0) for f in all_files])
C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:22: DtypeWarning: Columns (13,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv(f, encoding_errors='ignore', header=0) for f in all_files])
C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:22: DtypeWarning: Columns (13,32) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv(f, encoding_errors='ignore', header=0) for f in all_files])
C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:22: DtypeWarning: Columns (34,35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat([pd.read_csv(f, encodi

(1755610, 14)


C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BusinessType'] = data['BusinessType'].fillna('Individual')
C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['InitialInterestRate'] = data['InitialInterestRate'].fillna(data['InitialInterestRate'].min())
C:\Users\sebas\AppData\Local\Temp\ipykernel_21868\53397800.py:44: SettingWithCopyWarning: 
A value is trying to be set on a

,BankName,GrossApproval,ApprovalDate,ApprovalFiscalYear,InitialInterestRate,TermInMonths,NaicsCode,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,GrossChargeOffAmount,RevolverStatus,JobsSupported
0,CIBM Bank,330000.0,10/01/1990,1991,0.0,84,421990.0,CORPORATION,NaN,PIF,07/31/2005,0,0,0
1,BMO Harris Bank National Association,98900.0,10/01/1990,1991,0.0,11,NaN,CORPORATION,NaN,PIF,07/31/2005,0,0,0
2,MISSINGMAINBANKID,120000.0,10/01/1990,1991,0.0,240,421830.0,CORPORATION,NaN,CANCLD,NaN,0,0,0
3,"U.S. Bank, National Association",250000.0,10/01/1990,1991,0.0,48,NaN,CORPORATION,NaN,PIF,07/31/2005,0,0,0
4,Manufacturers and Traders Trust Company,33500.0,10/01/1990,1991,0.0,84,NaN,CORPORATION,NaN,PIF,07/31/2005,0,0,0


In [53]:
#get percentage of rows since 2018
print(pruned_data[pruned_data['ApprovalFiscalYear'] >= 2018].shape[0]/pruned_data.shape[0])

0.1678789708420435


In [54]:
#get percentage of revolving credit lines
print(pruned_data[pruned_data['RevolverStatus'] == 1].shape[0]/pruned_data.shape[0])

0.2525327379087611


In [64]:
#get minimum interest rate for each fiscal year
min_intrest_map = pruned_data[['ApprovalFiscalYear', 'InitialInterestRate']].dropna().groupby('ApprovalFiscalYear').mean().to_dict()
#replace 0's in dictionary with minium interest rate from dictionary
min_value = min([i for i in min_intrest_map.values() if i>0])
min_intrest_map = {k: v if v != 0 else min_value for k, v in min_intrest_map.items()}
print(min_intrest_map)

TypeError: '>' not supported between instances of 'dict' and 'int'

In [55]:
#get each columns percentage of null values
null_percent = pruned_data.isnull().sum() / len(pruned_data)
null_percent = null_percent[null_percent > 0]
null_percent.sort_values(inplace=True)
null_percent

NaicsCode         0.116069
PaidInFullDate    0.395203
BusinessAge       0.832543
dtype: float64